In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/aimprove-contest-week-2/Test-set-AImprove-2- detailed.csv
/kaggle/input/aimprove-contest-week-2/Test-set-AImprove-2..csv
/kaggle/input/aimprove-contest-week-2/Train-set-AImprove2.csv


In [2]:
df = pd.read_csv('/kaggle/input/aimprove-contest-week-2/Train-set-AImprove2.csv')
test = pd.read_csv('/kaggle/input/aimprove-contest-week-2/Test-set-AImprove-2- detailed.csv')

In [3]:
ch = 'T'
df = df[~df.apply(lambda row : any(str(ch) in str(cell) for cell in row), axis = 1)]
df.reset_index(drop = True, inplace = True)

In [4]:
char = '-'
rechar = '0'
df = df.applymap(lambda x : str(x).replace(char, rechar))

In [5]:
target = df.PrecipitationSumInches

In [6]:
df['DewPointHighF'] = pd.to_numeric(df['DewPointHighF'])
df['DewPointAvgF'] = pd.to_numeric(df['DewPointAvgF'])
df['DewPointLowF'] = pd.to_numeric(df['DewPointLowF'])
df['HumidityHighPercent'] = pd.to_numeric(df['HumidityHighPercent'])
df['HumidityAvgPercent'] = pd.to_numeric(df['HumidityAvgPercent'])
df['HumidityLowPercent'] = pd.to_numeric(df['HumidityLowPercent'])
df['SeaLevelPressureHighInches'] = pd.to_numeric(df['SeaLevelPressureHighInches'])
df['SeaLevelPressureAvgInches'] = pd.to_numeric(df['SeaLevelPressureAvgInches'])
df['SeaLevelPressureLowInches'] = pd.to_numeric(df['SeaLevelPressureLowInches'])
df['VisibilityHighMiles'] = pd.to_numeric(df['VisibilityHighMiles'])
df['VisibilityAvgMiles'] = pd.to_numeric(df['VisibilityAvgMiles'])
df['VisibilityLowMiles'] = pd.to_numeric(df['VisibilityLowMiles'])
df['WindHighMPH'] = pd.to_numeric(df['WindHighMPH'])
df['WindAvgMPH'] = pd.to_numeric(df['WindAvgMPH'])
df['WindGustMPH'] = pd.to_numeric(df['WindGustMPH'])

In [7]:
df.drop(columns = ['PrecipitationSumInches', 'Date', 'Events'], axis = 1, inplace = True)

In [8]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [9]:
X_train_full, X_valid_full, y_train, y_valid = train_test_split(df, target, train_size = 0.8, test_size = 0.2, random_state = 22)

In [10]:
rf = RandomForestRegressor(n_estimators = 42, random_state = 21)
rf.fit(X_train_full, y_train)

RandomForestRegressor(n_estimators=42, random_state=21)

In [11]:
pred = rf.predict(X_valid_full)
print(pred)
mse = mean_squared_error(y_valid, pred)
print('Mse', mse)

[0.00000000e+00 0.00000000e+00 9.52380952e-04 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 2.11190476e-01
 2.95238095e-02 4.76190476e-04 1.07142857e-02 3.57142857e-03
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 1.95238095e-02 0.00000000e+00 0.00000000e+00 4.76190476e-04
 0.00000000e+00 1.90476190e-03 0.00000000e+00 2.38095238e-04
 0.00000000e+00 2.38095238e-04 0.00000000e+00 0.00000000e+00
 0.00000000e+00 4.00000000e-02 1.00000000e-02 7.04761905e-02
 0.00000000e+00 1.34285714e-01 0.00000000e+00 0.00000000e+00
 7.14285714e-04 1.57619048e-01 4.76190476e-04 6.19047619e-03
 0.00000000e+00 1.42857143e-03 9.52380952e-04 0.00000000e+00
 0.00000000e+00 1.66666667e-03 0.00000000e+00 9.28571429e-03
 0.00000000e+00 0.00000000e+00 0.00000000e+00 1.20166667e+00
 4.84285714e-01 0.00000000e+00 0.00000000e+00 1.72380952e-01
 9.16666667e-02 2.00000000e-02 9.45238095e-02 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 4.76190476e-03
 0.00000000e+00 0.000000

In [12]:
col = df.columns
test_df = test.drop(columns = set(test.columns) - set(col), axis = 1)

In [13]:
prediction = rf.predict(test_df)

In [14]:
print(prediction)

[0.         0.00047619 0.00047619 0.16309524 0.         0.
 0.00047619 0.00047619 0.         0.04047619 0.         0.00190476
 0.         0.         0.         0.         0.        ]


In [15]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression

In [16]:
ans_df = pd.DataFrame({
    'Date' : test['Date'],
    'Predictions' : prediction
})
ans_df.to_csv('submission.csv', index = False)

In [17]:
print(ans_df)

          Date  Predictions
0   2017-07-12     0.000000
1   2017-07-13     0.000476
2   2017-07-14     0.000476
3   2017-07-15     0.163095
4   2017-07-16     0.000000
5   2017-07-18     0.000000
6   2017-07-19     0.000476
7   2017-07-20     0.000476
8   2017-07-22     0.000000
9   2017-07-23     0.040476
10  2017-07-24     0.000000
11  2017-07-25     0.001905
12  2017-07-27     0.000000
13  2017-07-28     0.000000
14  2017-07-29     0.000000
15  2017-07-30     0.000000
16  2017-07-31     0.000000
